<a href="https://colab.research.google.com/github/aquapathos/ColabHowto/blob/master/RealTime_Rendering_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab 上で　リアルタイムに処理結果動画をレンダリングする一方法

![battlezone](https://user-images.githubusercontent.com/5820803/100510012-7f523b00-31b1-11eb-92a3-379ed77bdb2f.gif)


In [ ]:
%%bash
wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null
unzip ngrok-stable-linux-amd64.zip
pip install bottle > /dev/null
pip install bottle_websocket > /dev/null
pip install gevent > /dev/null

# ランタイム再起動
一度ランタイムを再起動すること。

上のセルでインストールしたライブラリは、ランタイム起動時にはなかったものなので動作しないため。


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js

def use_cam(url, quality=0.8):
  print()  # このプリント文は必須
  js = Javascript('''
    async function useCam(url, quality) {
      
      const div = document.createElement('div');
      document.body.appendChild(div);
      
      const canvas = document.createElement('canvas');
      canvas.width  = 240;
      canvas.height = 315;
      const canvasCtx = canvas.getContext('2d');
      div.appendChild(canvas);

      //exit button　（効かないです）
      const btn_div = document.createElement('div');
      document.body.appendChild(btn_div);
      const exit_btn = document.createElement('button');
      exit_btn.textContent = 'Exit';
      var exit_flg = false
      exit_btn.onclick = function() {exit_flg = true};
      btn_div.appendChild(exit_btn);

      //log
      let jsLog = function(abc) {
        document.querySelector("#output-area").appendChild(document.createTextNode(`${abc} `));
      }
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      //for websocket connection.
      var connection = 0
      var flag_count = 0
      var send_flg = false

      // loop
      _canvasUpdate();
      async function _canvasUpdate() {
            flag_count += 1
            
            // 少し（２００カウント）待ってから WebSocket を実行
            if (flag_count == 200){
                print("200")
                connection = new WebSocket(url); 
                jsLog("Connect_start")
            }
            if (flag_count == 300){
                print("300")
                connection.onmessage = function(e) {
                    var image = new Image()
                    image.src = e.data;
                    image.onload = function(){canvasCtx.drawImage(image, 0, 0, 160, 210, 0, 0, canvas.width, canvas.height)}
                    send_flg=false
                };
                jsLog("Set_recieve")
            }
            if(flag_count > 400){
                const img = canvas.toDataURL('image/jpeg', quality);
                
                if (send_flg==false){
                    connection.send(img);
                    send_flg = true
                }
            }
          if (!exit_flg){
              requestAnimationFrame(_canvasUpdate);   
          }else{
              //stream.getVideoTracks()[0].stop();
              connection.close();
          }
      };
    }
    ''')
  display(js)
  data = eval_js('useCam("{}", {})'.format(url, quality))

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')
"""
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json;url = json.load(sys.stdin)['tunnels'][0]['public_url'];print(url);f = open('url.txt', 'w');f.write(url);f.close()"
"""
import subprocess
url = "wss"
while len(url)==3:
  result = subprocess.run(["curl -s http://localhost:4040/api/tunnels | python3 -c \"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])\""], encoding='utf-8',shell=True, stdout=subprocess.PIPE)
  url = "wss"+result.stdout[5:-1]
url

In [ ]:
import numpy as np
import cv2
import json
import bottle
import gevent
from bottle.ext.websocket import GeventWebSocketServer
from bottle.ext.websocket import websocket
from multiprocessing import Pool
from PIL import Image
from io import BytesIO
import base64
import gym

env = gym.make('BattleZone-v0') 

socket = bottle.Bottle()
@socket.route('/', apply=[websocket])
def wsbin(ws):
    kbreak = False
    while not kbreak:
        done = False
        env.reset()
        while not done:
            try:
                action = env.action_space.sample()                  # ランダムな行動選択
                out_img, reward, done, info = env.step(action)      # １ステップ実行
                if done: 
                  break
            
                #encode to string
                _, encimg = cv2.imencode(".jpg", out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
                img_str = encimg.tostring()
                img_str = "data:image/jpeg;base64," + base64.b64encode(img_str).decode('utf-8')
                ws.send(img_str)
            except:
                kbreak = True
                done = True
                pass
                #print("error")

if __name__ == '__main__':
    # prepare multiprocess
    _pool = Pool(processes=2)
    _pool.apply_async(use_cam, (url, 0.8))
    socket.run(host='0.0.0.0', port=6006, server=GeventWebSocketServer)
   